In [3]:
#Install
!pip install requests pandas numpy

Defaulting to user installation because normal site-packages is not writeable


In [8]:
#Import
import requests
import pandas as pd
import numpy as np
import http.client, json
from urllib.parse import urlparse
import json

## Premières requêtes

In [11]:
def get_data(url):
    parsed_url = urlparse(url)
    conn = http.client.HTTPConnection(parsed_url.netloc)
    conn.request("GET", parsed_url.path)
    res = conn.getresponse()
    data = res.read()
    jsondata = json.loads(data.decode("utf-8"))
    return jsondata

idée maintenant : créer un dictionnaire : 
- key=joueur(id), 
- value = dict= (key=matchs joués par le joueur, value=statistics)

https://www.sofascore.com/api/v1/unique-tournament/17/season/52186/events/last/0
-> donne les id des matchs du championnat 17 sur la saisons 52186

https://www.sofascore.com/api/v1/event/12436466/lineups
-> donne les stats par joueur du match avec l'id 12436466

## Exemple sur la saison 23-24
1) On recupère la liste des id des matchs de la saison
2) on boucle sur les matchs
    -> sur chaque match, on ajoute a notre dictionnaire : 
        -> joueur : stat du match

In [24]:
players_data = {}
matches_id = []

In [13]:
data = get_data('https://www.sofascore.com/api/v1/unique-tournament/17/season/52186/events/last/0')

In [15]:
data.keys()

dict_keys(['events', 'hasNextPage'])

In [16]:
data['hasNextPage']

True

In [21]:
for k,v in data['events'][0].items():
    print(k)
    print(v)
    print('\n')

tournament
{'name': 'Premier League', 'slug': 'premier-league', 'category': {'name': 'England', 'slug': 'england', 'sport': {'name': 'Football', 'slug': 'football', 'id': 1}, 'id': 1, 'country': {'alpha2': 'EN', 'alpha3': 'ENG', 'name': 'England', 'slug': 'england'}, 'flag': 'england', 'alpha2': 'EN'}, 'uniqueTournament': {'name': 'Premier League', 'slug': 'premier-league', 'primaryColorHex': '#3c1c5a', 'secondaryColorHex': '#f80158', 'category': {'name': 'England', 'slug': 'england', 'sport': {'name': 'Football', 'slug': 'football', 'id': 1}, 'id': 1, 'country': {'alpha2': 'EN', 'alpha3': 'ENG', 'name': 'England', 'slug': 'england'}, 'flag': 'england', 'alpha2': 'EN'}, 'userCount': 1378471, 'hasPerformanceGraphFeature': True, 'id': 17, 'country': {}, 'hasEventPlayerStatistics': True, 'displayInverseHomeAwayTeams': False, 'fieldTranslations': {'nameTranslation': {'ar': 'الدوري الإنجليزي الممتاز', 'hi': 'प्रिमियर लीग'}, 'shortNameTranslation': {}}}, 'priority': 617, 'isGroup': False, 'i

In [25]:
url_root = 'https://www.sofascore.com/api/v1/unique-tournament/17/season/52186/events/last/'
hasNextPage = True
i = 0

while hasNextPage:
    present_url = url_root + str(i)
    data = get_data(present_url)
    for match in data['events']:
        matches_id.append(match['id'])
    hasNextPage = data['hasNextPage'] #True or False
    i+=1 #access to nextpage

In [27]:
len(matches_id)

380

Il y a 38 journées de championnat et 10 match par journée donc 380 est le bon résultat